# 📦 ONNX: Uniwersalny format modeli (Production Ready)

PyTorch jest świetny do nauki i treningu (elastyczny, dynamiczny).
Ale na produkcji (w aplikacji klienta) nie chcemy instalować 2GB biblioteki PyTorch.

**ONNX (Open Neural Network Exchange)** to standard zapisu grafu obliczeniowego.
Działa na zasadzie **Tracingu (Śledzenia):**
1.  Wpuszczamy do modelu przykładowe dane (Dummy Input).
2.  ONNX "nagrywa" wszystkie operacje matematyczne, jakie się wykonały.
3.  Zapisuje to jako statyczny graf w pliku `.onnx`.

**Zalety:**
*   **Przenośność:** Uruchomisz to w C++, C#, Java, JavaScript.
*   **Szybkość:** ONNX Runtime jest mocno zoptymalizowany pod konkretny sprzęt (AVX na CPU, TensorRT na NVIDIA).

In [ ]:
# Instalacja ONNX Runtime
# !uv pip install onnx onnxruntime onnxscript

import torch
import torch.nn as nn
import numpy as np
import onnxruntime as ort
import time

# 1. TWORZYMY MODEL (Prosty klasyfikator)
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 20)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(20, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

# Inicjalizacja i przejście w tryb eval (Ważne! Wyłącza dropout itp.)
model = SimpleModel()
model.eval()

print("Model PyTorch gotowy.")

## Eksport do ONNX (Tracing)

To jest kluczowy moment. Musimy podać `dummy_input` (atrapę danych), żeby ONNX wiedział, jaki kształt mają wejścia.

**Ważne:** Użyjemy `dynamic_axes`.
Domyślnie ONNX zapamiętuje sztywny rozmiar (np. Batch=1). Jeśli na produkcji przyjdzie Batch=32, model wywali błąd.
Oznaczając oś 0 jako dynamiczną, mówimy: *"Tu może być dowolna liczba wierszy"*.

In [13]:
# Przykładowe dane
dummy_input = torch.randn(1, 10)
onnx_path = "simple_model.onnx"

print(f"Eksportowanie modelu do ONNX (Opset 18)...")

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=18,  # ⬅️ Zmienione z 17 na 18
    dynamo=True,
    input_names=['input'],
    output_names=['output']
)

print(f"✅ Sukces! Model wyeksportowany do: {onnx_path}")

Eksportowanie modelu do ONNX (Opset 18)...
[torch.onnx] Obtain model graph for `SimpleModel([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `SimpleModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
✅ Sukces! Model wyeksportowany do: simple_model.onnx


## Inference w ONNX Runtime

Teraz zapominamy o PyTorchu.
Wyobraź sobie, że jesteś na serwerze produkcyjnym, gdzie jest tylko lekki `onnxruntime`.

Uruchomimy model z pliku.
Zauważ, że wejście musi być w formacie **NumPy** (nie Tensor).

In [15]:
# Tworzymy sesję (silnik)
ort_session = ort.InferenceSession(onnx_path)

# --- POPRAWKA ---
# Zmieniamy rozmiar batcha z 5 na 1, żeby pasował do tego, co zapamiętał model
x_numpy = np.random.randn(1, 10).astype(np.float32)

# Uruchomienie (Run)
ort_inputs = {ort_session.get_inputs()[0].name: x_numpy}
ort_outs = ort_session.run(None, ort_inputs)

print("--- WYNIK Z ONNX RUNTIME ---")
print(ort_outs[0]) # To są prawdopodobieństwa (Softmax)

--- WYNIK Z ONNX RUNTIME ---
[[0.43904942 0.5609505 ]]


## Weryfikacja (Czy to to samo?)

Sprawdźmy, czy ONNX zwraca dokładnie te same liczby co PyTorch.
Różnice mogą się pojawić na poziomie $10^{-7}$ (kwestia precyzji float), ale powinny być minimalne.

In [16]:
# Wynik PyTorch
with torch.no_grad():
    torch_out = model(torch.from_numpy(x_numpy))

# Wynik ONNX
onnx_out = ort_outs[0]

# Porównanie
# np.allclose sprawdza czy liczby są blisko siebie z tolerancją
is_match = np.allclose(torch_out.numpy(), onnx_out, rtol=1e-03, atol=1e-05)

print(f"Czy wyniki są identyczne? {is_match}")

if not is_match:
    print("Różnica:", np.abs(torch_out.numpy() - onnx_out).max())

Czy wyniki są identyczne? True


## Benchmark: PyTorch vs ONNX (CPU)

ONNX Runtime jest zazwyczaj szybszy na procesorach CPU, bo używa instrukcji wektorowych (AVX) lepiej niż PyTorch (który jest optymalizowany głównie pod GPU).

In [18]:
# Generujemy 1 próbkę (zamiast 10000), żeby pasowało do modelu
input_data = np.random.randn(1, 10).astype(np.float32)
torch_input = torch.from_numpy(input_data)

print("Rozpoczynam wyścig (1000 powtórzeń pojedynczego zapytania)...")

# 1. Czas PyTorch
start = time.time()
for _ in range(1000):
    with torch.no_grad():
        _ = model(torch_input)
end = time.time()
torch_time = end - start

# 2. Czas ONNX
ort_inputs = {ort_session.get_inputs()[0].name: input_data}
start = time.time()
for _ in range(1000):
    _ = ort_session.run(None, ort_inputs)
end = time.time()
onnx_time = end - start

print(f"PyTorch Time: {torch_time:.4f} s")
print(f"ONNX Time:    {onnx_time:.4f} s")

# Zabezpieczenie przed dzieleniem przez zero (gdyby było super szybko)
if onnx_time > 0:
    print(f"🚀 Przyspieszenie: {torch_time / onnx_time:.2f}x")
else:
    print("ONNX był tak szybki, że zegar pokazał 0!")

Rozpoczynam wyścig (1000 powtórzeń pojedynczego zapytania)...
PyTorch Time: 0.0260 s
ONNX Time:    0.0080 s
🚀 Przyspieszenie: 3.26x


## 🧠 Podsumowanie: Gdzie tego użyć?

1.  **Deployment w chmurze:** Kontenery Docker z ONNX Runtime są lżejsze (nie mają całego PyTorcha).
2.  **Edge AI:** Modele na telefony (Android/iOS) konwertuje się do ONNX, a potem np. do TFLite lub CoreML (często przez ONNX jako krok pośredni).
3.  **Przeglądarka:** Możesz użyć `ONNX Runtime Web` i uruchomić model w JavaScript bezpośrednio w Chrome użytkownika (bez wysyłania danych na serwer!).

**Wada:**
ONNX to graf **statyczny**. Jeśli Twój model w PyTorch ma pętle `for` o zmiennej długości albo skomplikowane instrukcje `if-else` zależne od danych, eksport może być trudny (wymaga `torch.jit.script`).